In [1]:
import platform
platform.platform() #checking python for arm64 which is needed

'macOS-13.0.1-arm64-arm-64bit'

In [2]:
import torch
torch.__version__ #checking pytorch version which must be >= 12.3

'1.13.0'

In [3]:
import transformers

In [4]:
torch.has_mps

True

In [5]:
from datasets import load_dataset

trec=load_dataset('trec', split='train[:64]')
trec

Found cached dataset trec (/Users/tanya.banerjee/.cache/huggingface/datasets/trec/default/2.0.0/f2469cab1b5fceec7249fda55360dfdbd92a7a5b545e91ea0f78ad108ffac1c2)


Dataset({
    features: ['text', 'coarse_label', 'fine_label'],
    num_rows: 64
})

In [6]:
trec[0]

{'text': 'How did serfdom develop in and then leave Russia ?',
 'coarse_label': 2,
 'fine_label': 26}

In [7]:
from transformers import AutoModel, AutoTokenizer

In [8]:
tokenizer=AutoTokenizer.from_pretrained('bert-base-uncased') #get tokenizer

In [9]:
tokens=tokenizer(trec['text'], max_length=512, truncation=True, padding=True) #tokenize

In [10]:
device=torch.device('mps') #the variable device means the Apple M1 GPU now
device

device(type='mps')

In [11]:
tokens #seeing what the tokenizer did

{'input_ids': [[101, 2129, 2106, 14262, 2546, 9527, 4503, 1999, 1998, 2059, 2681, 3607, 1029, 102, 0, 0, 0, 0, 0, 0, 0], [101, 2054, 3152, 2956, 1996, 2839, 4831, 6672, 11294, 1029, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 2129, 2064, 1045, 2424, 1037, 2862, 1997, 12330, 1005, 2613, 3415, 1029, 102, 0, 0, 0, 0, 0, 0, 0], [101, 2054, 1042, 5004, 2140, 13273, 1996, 17763, 2044, 1996, 2822, 2095, 1997, 1996, 10608, 1029, 102, 0, 0, 0, 0], [101, 2054, 2003, 1996, 2440, 2433, 1997, 1012, 4012, 1029, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 2054, 17152, 7028, 8040, 28819, 16570, 10312, 1996, 8513, 2013, 2026, 6265, 1029, 102, 0, 0, 0, 0, 0, 0], [101, 2054, 2136, 2106, 3598, 1005, 1055, 2358, 1012, 3434, 13240, 2468, 1029, 102, 0, 0, 0, 0, 0, 0, 0], [101, 2054, 2003, 1996, 4587, 9518, 1029, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 2054, 2024, 11290, 16285, 1029, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 2171, 1996, 11228, 1011, 4320, 17284, 4477, 1997, 1996, 2214, 2225

In [12]:
from transformers import BertForSequenceClassification, BertConfig

In [13]:
config=BertConfig.from_pretrained('bert-base-uncased')
config.num_labels=6 #6 options in the coarse-grained labels
model=BertForSequenceClassification(config).to(device)

In [14]:
optim = torch.optim.AdamW(model.parameters(), lr=5e-5)

In [15]:
print(model.named_parameters)

<bound method Module.named_parameters of BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNo

In [16]:
for name, param in model.named_parameters():
    if 'classifier' not in name: # fix all layers except classifier layer
        param.requires_grad = False
    else:
        print('not me')

not me
not me


In [17]:
import numpy as np

# initialize array to be used
labels = np.zeros(
    (len(trec), max(trec['coarse_label'])+1)
)
# one-hot encode
labels[np.arange(len(trec)), trec['coarse_label']] = 1
labels[:5]
type(labels[0][0])

numpy.float64

In [18]:
labels = torch.Tensor(labels)

In [19]:
class TrecDataset(torch.utils.data.Dataset):
    def __init__(self, tokens, labels):
        self.tokens = tokens
        self.labels = labels

    def __getitem__(self, idx):
        input_ids = self.tokens[idx].ids
        attention_mask = self.tokens[idx].attention_mask
        labels = self.labels[idx]
        return {
            'input_ids': torch.tensor(input_ids),
            'attention_mask': torch.tensor(attention_mask),
            'labels': torch.tensor(labels)
        }

    def __len__(self):
        return len(self.labels)

dataset = TrecDataset(tokens, labels)

In [20]:
loader = torch.utils.data.DataLoader(
    dataset, batch_size=4
)

In [21]:
model.train() #activate training mode

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [22]:
for batch in loader:
    #move to device
    batch_mps = {
        'input_ids': batch['input_ids'].to(device),
        'attention_mask': batch['attention_mask'].to(device),
        'labels': batch['labels'].to(device)
    }
    # initialize calculated gradients (from prev step)
    optim.zero_grad()
    # train model on batch and return outputs (incl. loss)
    outputs = model(**batch_mps)
    # extract loss
    loss = outputs[0]
    # calculate loss for every parameter that needs grad update
    loss.backward()
    # update parameters
    optim.step()

/var/folders/ht/vj5_hz494059tyzm_9_1x92c0000gn/T/ipykernel_5299/3304426640.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'labels': torch.tensor(labels)


In [23]:
test=load_dataset('trec', split='train[64:100]')

Found cached dataset trec (/Users/tanya.banerjee/.cache/huggingface/datasets/trec/default/2.0.0/f2469cab1b5fceec7249fda55360dfdbd92a7a5b545e91ea0f78ad108ffac1c2)


In [24]:
tokens_test=tokenizer(test['text'], max_length=512, truncation=True, padding=True) #tokenize

In [25]:
proper_labels = np.zeros(
    (len(test), max(test['coarse_label'])+1), dtype='float32'
)
# one-hot encode
proper_labels[np.arange(len(test)), test['coarse_label']] = 1
proper_labels[:5]

array([[0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.]], dtype=float32)

In [26]:
test_dataset = TrecDataset(tokens_test, proper_labels)

In [27]:
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=1
)

In [28]:
total_loss=torch.Tensor([0]).to(device)
for batch in test_loader:
    dicty = {
        'input_ids': batch['input_ids'].to(device),
        'attention_mask': batch['attention_mask'].to(device),
        'labels': batch['labels'].to(device)
    }
    outputs = model(**dicty)
    loss = outputs[0]
    total_loss=total_loss+loss
    
print(total_loss)

tensor([22.6876], device='mps:0', grad_fn=<AddBackward0>)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/_tensor_str.py:115: UserWarning: The operator 'aten::nonzero' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:11.)
  nonzero_finite_vals = torch.masked_select(
